From https://huggingface.co/docs/transformers/en/training

Some other useful links:

https://colab.research.google.com/drive/1U7SX7jNYsNQG5BY1xEQQHu48Pn6Vgnyt?usp=sharing

https://huggingface.co/docs/transformers/en/main_classes/output

https://towardsdatascience.com/sigmoid-and-softmax-functions-in-5-minutes-f516c80ea1f9/

In [2]:
from datasets import load_dataset
dataset = load_dataset('yelp_review_full')

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
dataset['train'][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-cased')

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/Users/gordonpetty/opt/anaconda3/envs/ernie/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [6]:
tokenized_dataset = dataset.map(tokenize_function)

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(1000))

In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5, torch_dtype="auto")

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="test_trainer")

In [16]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [18]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [19]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.173383,0.516000
2,No log,1.033741,0.554000
3,No log,1.045023,0.569000


TrainOutput(global_step=375, training_loss=1.0462149251302084, metrics={'train_runtime': 1288.6583, 'train_samples_per_second': 2.328, 'train_steps_per_second': 0.291, 'total_flos': 789354427392000.0, 'train_loss': 1.0462149251302084, 'epoch': 3.0})

In [25]:
# Save trained model locally
save_path = 'test_trainer/trained_model'
trainer.save_model(save_path)

In [33]:
trainer.predict(small_eval_dataset)

PredictionOutput(predictions=array([[-2.2285388 , -2.2718303 , -1.4508818 ,  1.697949  ,  3.0408947 ],
       [-3.0135174 , -0.8645506 ,  2.11775   ,  2.2628431 , -0.731693  ],
       [ 3.134579  ,  0.8743686 , -0.70214695, -1.4437107 , -0.9419497 ],
       ...,
       [-2.8668244 , -2.0274973 , -0.07909125,  2.1525335 ,  1.1882511 ],
       [-1.6088244 ,  1.6698655 ,  2.290367  , -0.03984155, -1.9289058 ],
       [-0.20784232,  1.8755993 ,  1.1396902 , -0.5347756 , -1.6096087 ]],
      dtype=float32), label_ids=array([2, 4, 1, 4, 3, 4, 2, 3, 2, 3, 0, 0, 3, 2, 2, 1, 3, 1, 2, 2, 1, 2,
       3, 1, 1, 3, 4, 0, 0, 2, 2, 2, 1, 3, 4, 0, 0, 1, 3, 2, 0, 2, 0, 0,
       3, 0, 3, 2, 3, 0, 1, 1, 3, 3, 4, 4, 1, 4, 1, 3, 1, 0, 0, 1, 4, 1,
       4, 3, 2, 4, 1, 0, 3, 3, 4, 1, 2, 1, 0, 4, 4, 4, 2, 3, 3, 1, 4, 0,
       4, 2, 3, 0, 0, 0, 3, 4, 0, 0, 1, 4, 4, 0, 0, 1, 1, 0, 4, 2, 2, 1,
       1, 4, 0, 4, 0, 3, 2, 0, 4, 4, 4, 2, 0, 0, 0, 1, 3, 0, 2, 0, 3, 2,
       2, 2, 0, 3, 4, 3, 0, 1, 0, 1, 0, 0, 4

In [34]:
trainer.evaluate()

{'eval_loss': 1.0450234413146973,
 'eval_accuracy': 0.569,
 'eval_runtime': 101.3841,
 'eval_samples_per_second': 9.863,
 'eval_steps_per_second': 1.233,
 'epoch': 3.0}

In [44]:
# Create some fake reviews and see how they are classified by our model
reviews = [
    'This was the absolute best restaurant I have ever been to!',
    'I went here and it was just OK. The food was mediocre but the service was good',
    'My experience here was absolutely awful, and I will never return!'
]

# Encode them!
review_encoded = tokenizer(reviews,padding="max_length", truncation=True, return_tensors='pt')
# Reformat for interpretation by model
review_encoded = {k: v.to(trainer.model.device) for k,v in review_encoded.items()}
review_encoded

In [48]:
outputs = trainer.model(**review_encoded)
# The outputs are Logits, which we then need to convert to our classifications
# Here we can simply 
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.8398, -1.5912, -1.4926,  0.5747,  2.7292],
        [-2.1735,  0.2691,  2.5914,  1.4665, -1.5088],
        [ 3.2254,  0.8062, -0.7483, -1.3968, -1.0237]], device='mps:0',
       grad_fn=<LinearBackward0>), hidden_states=None, attentions=None)

In [97]:
import torch
softmax = torch.nn.Softmax(dim=1)
logits = outputs.logits.squeeze().cpu()
sig_outputs = sigmoid(logits).detach().numpy()
logits_np = logits.detach().numpy()

In [94]:
np.round(sig_outputs,2)

array([[0.02, 0.01, 0.01, 0.1 , 0.85],
       [0.01, 0.07, 0.69, 0.22, 0.01],
       [0.88, 0.08, 0.02, 0.01, 0.01]], dtype=float32)

In [ ]:
np.argmax(logits)

In [102]:
categories = np.argmax(logits_np,axis=1)

In [101]:
mapping = {0:'1 star',1:'2 stars',2:'3 stars',3:'4 stars',4:'5 stars'}

In [105]:
classifications = [mapping[c] for c in categories]
classifications

['5 stars', '3 stars', '1 star']